In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model
import numpy as np

### Sequential vs Functional API

In [3]:
# Sequential API
'''
Use when the model is a simple stack of layers: one input → layers in a line → one output.
Pros:
- Very simple, readable
- Great for prototypes and straightforward architectures
Limitations:
- No branching, no multiple inputs/outputs, no complex graphs
- Harder to implement skip connections, attention, etc.
'''
# Stack example

model = models.Sequential([
    layers.Input(shape=(28 * 28,)),       # flattened image
    layers.Dense(128, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")  # 10 classes
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [6]:
#Functional API
'''
Use when you need flexible architectures:
- Multiple inputs / outputs
- Skip connections (ResNet-style)
- Shared layers
- Any graph-like topology
'''
# Stack example
inputs = layers.Input(shape=(28 * 28,))
x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [9]:
# Example with a skip connection
inputs = layers.Input(shape=(128, ))

x = layers.Dense(64, activation="relu")(inputs)
skip = x
x = layers.Dense(64, activation="relu")(x)
x = layers.Add()([x, skip])  # skip connection
outputs = layers.Dense(10, activation="softmax")(x)

model = Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

### Train / validation split

In [18]:
# Simple split with scikit-learn
from sklearn.model_selection import train_test_split

X = np.random.normal(loc=0, scale=1, size=(10000, 15))
y = np.random.randint(0, 2, 10000)

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,      # 20% for validation
    random_state=42,
    stratify=y)          # keep class proportions (classification)

# Stack example
inputs = layers.Input(shape=(15,))
x = layers.Dense(128, activation="relu")(inputs)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2)

Epoch 1/20
250/250 - 1s - 4ms/step - accuracy: 0.4762 - loss: 0.8628 - val_accuracy: 0.5030 - val_loss: 0.7142
Epoch 2/20
250/250 - 0s - 2ms/step - accuracy: 0.5132 - loss: 0.7015 - val_accuracy: 0.5000 - val_loss: 0.7078
Epoch 3/20
250/250 - 0s - 2ms/step - accuracy: 0.5226 - loss: 0.6954 - val_accuracy: 0.4915 - val_loss: 0.7080
Epoch 4/20
250/250 - 0s - 2ms/step - accuracy: 0.5360 - loss: 0.6913 - val_accuracy: 0.4995 - val_loss: 0.7232
Epoch 5/20
250/250 - 0s - 2ms/step - accuracy: 0.5520 - loss: 0.6857 - val_accuracy: 0.4965 - val_loss: 0.7046
Epoch 6/20
250/250 - 0s - 2ms/step - accuracy: 0.5623 - loss: 0.6812 - val_accuracy: 0.5150 - val_loss: 0.7032
Epoch 7/20
250/250 - 0s - 2ms/step - accuracy: 0.5795 - loss: 0.6753 - val_accuracy: 0.4965 - val_loss: 0.7098
Epoch 8/20
250/250 - 0s - 2ms/step - accuracy: 0.5964 - loss: 0.6677 - val_accuracy: 0.4845 - val_loss: 0.7314
Epoch 9/20
250/250 - 0s - 2ms/step - accuracy: 0.6014 - loss: 0.6611 - val_accuracy: 0.5005 - val_loss: 0.7181
E

model.fit: key parameters

- x, y – training data and labels.

- batch_size – number of samples per gradient step.

- epochs – how many passes over the training data.

- validation_data=(X_val, y_val) or validation_split=0.2

- callbacks=[...] – hooks called during training (see next section).

### Callbacks in Keras

**Callbacks** = objects that Keras calls at certain points during training:

- Start/end of epoch

- Start/end of batch

- When metric changes, etc.

They allow you to:

- Stop training early

- Save best model

- Change learning rate on the fly

- Log to TensorBoard

- Implement custom logic

**Common built-in callbacks**:

- EarlyStopping

- ModelCheckpoint

- ReduceLROnPlateau

- TensorBoard

- LearningRateScheduler

In [21]:
# Early Stopping Callback Example
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor="val_loss",       # what to monitor
    patience=3,               # epochs to wait without improvement
    restore_best_weights=True)# roll back to best epoch

checkpoint = ModelCheckpoint(
    filepath="best_model.keras",
    monitor="val_loss",
    save_best_only=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, checkpoint],
    verbose=2)

Epoch 1/50
125/125 - 0s - 2ms/step - accuracy: 0.7355 - loss: 0.5342 - val_accuracy: 0.5170 - val_loss: 0.7920
Epoch 2/50
125/125 - 0s - 2ms/step - accuracy: 0.7371 - loss: 0.5282 - val_accuracy: 0.5135 - val_loss: 0.7933
Epoch 3/50
125/125 - 0s - 2ms/step - accuracy: 0.7420 - loss: 0.5248 - val_accuracy: 0.5125 - val_loss: 0.8057
Epoch 4/50
125/125 - 0s - 2ms/step - accuracy: 0.7449 - loss: 0.5188 - val_accuracy: 0.5150 - val_loss: 0.8119
